In [115]:
import pandas as pd
field_data = pd.read_csv('toy_fields.csv') #data about the size of the fields in acres
org_data = pd.read_csv('toy_orgs.csv') #data about the names of the orgs and their unique id's
yield_data = pd.read_csv('toy_yields.csv') #data about the harvest details 
#field_data.info()
#org_data.info()
#yield_data.info()
yield_data.head()

,crop_type,year,harvest_product,total_harvested,harvested_unit,planted_area,harvested_area,field_id,org_id
0,CROP.CORN.COMMERCIAL,2016,grain,53290.628852,bu,40.380000,560.0,cb1e3bd821dd673a0c378b16ba97075919aa993a,0966e34316e39217e4f26fbe20fc14e24c26567b
1,CROP.CORN.COMMERCIAL,2014,grain,40073.821554,bu,NaN,NaN,58d04a7094c1b08e9856932a02104d2a36b7b8ce,726cce9d9800d792d1e0fbb83923ac4c6be4f668
2,CROP.CORN.COMMERCIAL,2015,grain,45528.033941,bu,148.610000,NaN,5e3babe7b318f4245ceb16718665af0b54335443,598d17ec200eeb9e27ae829746ccf47c57761bc1
3,CROP.SOYBEAN.COMMERCIAL,2015,grain,13927.332280,bu,25.000000,NaN,02dc3fe6f889f3ece8daf456d26464d381bc9e55,fe62dee029474d52346542845674529025594056
4,CROP.CORN.COMMERCIAL,2014,grain,59676.634874,bu,5.026942,NaN,109e500b323e0ef7baadd4409830c7619d8cfbfa,fe62dee029474d52346542845674529025594056


#### hm. right away I can see some NaN values that warrant some attention. The missing entries tells me that the quality of this data quality is imperfect. I'll have to decide later on whether I want to discard those specific data rows or fill them in somehow. Also, another point of concern is the planted area and harvested area values. In the 1st row, 'planted_area' should not exceed the value of 'harvested_area'. The quality of this data may prove to be invalid should these entries are unable to be handled. 

In [176]:
updated_harvest_area = []
updated_planted_area = []
for index,row in yield_data.iterrows():
    if row['planted_area']==float('nan')or row['harvested_area']>row['planted_area']:
        updated_harvest_area.append(None)
        updated_planted_area.append(None)
    else:
        updated_harvest_area.append(row['harvested_area'])
        updated_planted_area.append(row['planted_area'])
         
yield_data['cleaned_planted_area'] = updated_planted_area
yield_data['cleaned_harvest_area'] = updated_harvest_area  
yield_data[:5]

,crop_type,year,harvest_product,total_harvested,harvested_unit,planted_area,harvested_area,cleaned_planted_area,cleaned_harvest_area,field_id,org_id
38,CROP.CORN.COMMERCIAL,2016,grain,127701.071130,bu,78.12,245.0,NaN,NaN,66dec081f750e2d33394a88a4436242bfbdcf6da,0966e34316e39217e4f26fbe20fc14e24c26567b
291,CROP.CORN.COMMERCIAL,2016,grain,25602.606473,bu,NaN,NaN,NaN,NaN,4da7eb451613f4e33463ea474b2d24f51d0540ff,0966e34316e39217e4f26fbe20fc14e24c26567b
307,CROP.SOYBEAN.COMMERCIAL,2016,grain,8171.547260,bu,NaN,25.0,NaN,25.0,de5bf9fd04e342823175dba3e765c067b1e28edf,0966e34316e39217e4f26fbe20fc14e24c26567b


#### With the above forloop, I create a new column called 'updated_harvest_area' to clean the data of harvest areas that exceeded the planted area

In [144]:
yield_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 2 to 416
Data columns (total 11 columns):
crop_type               415 non-null object
year                    415 non-null int64
harvest_product         415 non-null object
total_harvested         415 non-null float64
harvested_unit          415 non-null object
planted_area            304 non-null float64
harvested_area          207 non-null float64
field_id                415 non-null object
org_id                  415 non-null object
cleaned_planted_area    228 non-null float64
cleaned_harvest_area    131 non-null float64
dtypes: float64(5), int64(1), object(5)
memory usage: 35.7+ KB


#### There were many missing values in the planted area column and the harvested area column.  After the data cleaning, 77 of both the planted_area and harvest_area entries were cleared and only (228,131) respective valid entries are remaining. 

In [172]:
org_data.rename(columns = {'new_org_id': 'org_id'}, inplace= True)
org_data.columns
cols = yield_data.columns.tolist() # just rearranging column to make is easier to analyze
cols= cols[:5]+cols[7:]+ cols[5:7]
yield_data = yield_data[cols]
yield_data = yield_data.loc[yield_data['org_id']=='0966e34316e39217e4f26fbe20fc14e24c26567b']
yield_data.loc[yield_data['year']==2016]

,crop_type,year,harvest_product,total_harvested,harvested_unit,planted_area,harvested_area,cleaned_planted_area,cleaned_harvest_area,field_id,org_id
38,CROP.CORN.COMMERCIAL,2016,grain,127701.071130,bu,78.12,245.0,NaN,NaN,66dec081f750e2d33394a88a4436242bfbdcf6da,0966e34316e39217e4f26fbe20fc14e24c26567b
291,CROP.CORN.COMMERCIAL,2016,grain,25602.606473,bu,NaN,NaN,NaN,NaN,4da7eb451613f4e33463ea474b2d24f51d0540ff,0966e34316e39217e4f26fbe20fc14e24c26567b
307,CROP.SOYBEAN.COMMERCIAL,2016,grain,8171.547260,bu,NaN,25.0,NaN,25.0,de5bf9fd04e342823175dba3e765c067b1e28edf,0966e34316e39217e4f26fbe20fc14e24c26567b


#### we have to rename the 'organization_id' column in our toy_org.csv file to 'org_id' to be able to merge the two data frames

In [147]:
mergedDf = pd.merge(yield_data,org_data, left_on = ['org_id'],right_on = ['org_id'])
mergedDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 415 entries, 0 to 414
Data columns (total 13 columns):
crop_type               415 non-null object
year                    415 non-null int64
harvest_product         415 non-null object
total_harvested         415 non-null float64
harvested_unit          415 non-null object
field_id                415 non-null object
org_id                  415 non-null object
cleaned_planted_area    228 non-null float64
cleaned_harvest_area    131 non-null float64
planted_area            304 non-null float64
harvested_area          207 non-null float64
grower                  415 non-null object
organization_id         415 non-null object
dtypes: float64(5), int64(1), object(7)
memory usage: 45.4+ KB


In [149]:
mergedDf[:5]

,crop_type,year,harvest_product,total_harvested,harvested_unit,field_id,org_id,cleaned_planted_area,cleaned_harvest_area,planted_area,harvested_area,grower,organization_id
0,CROP.CORN.COMMERCIAL,2015,grain,45528.033941,bu,5e3babe7b318f4245ceb16718665af0b54335443,598d17ec200eeb9e27ae829746ccf47c57761bc1,148.61,NaN,148.61,NaN,Simmons Farm,1a40b159-81f6-4019-a6db-6af405a748d7
1,CROP.SOYBEAN.COMMERCIAL,2015,grain,30341.901524,bu,ed7e44f184a071a25c905e1ee9f73c6741e0bd01,598d17ec200eeb9e27ae829746ccf47c57761bc1,80.21,NaN,80.21,NaN,Simmons Farm,1a40b159-81f6-4019-a6db-6af405a748d7
2,CROP.CORN.COMMERCIAL,2015,grain,50987.861543,bu,fd72b33016dc565f97e287594573282d0ae29f08,598d17ec200eeb9e27ae829746ccf47c57761bc1,179.12,NaN,179.12,NaN,Simmons Farm,1a40b159-81f6-4019-a6db-6af405a748d7
3,CROP.SOYBEAN.COMMERCIAL,2015,grain,13685.647874,bu,6682442ec01eee124d9a2b40f049d2036596b969,598d17ec200eeb9e27ae829746ccf47c57761bc1,133.62,NaN,133.62,NaN,Simmons Farm,1a40b159-81f6-4019-a6db-6af405a748d7
4,CROP.SOYBEAN.COMMERCIAL,2017,grain,30074.126012,bu,daeab438051b9d0a654659b7f038f08b44c184af,598d17ec200eeb9e27ae829746ccf47c57761bc1,NaN,NaN,52.47,56.0,Simmons Farm,1a40b159-81f6-4019-a6db-6af405a748d7


In [125]:
mergedDf.grower.value_counts()

Simmons Farm        148
Cavanaugh Farm       82
Dudley Farm          51
Austin Farm          50
Depner Farm          34
Konecny Farm          8
Jenkins Farm          7
Odell Farm            6
Hawker Farm           5
Ross Farm             4
Gregory Farm          4
Overholt Farm         3
Flanagan Farm         3
Sifuentes Farm        2
Mull Farm             2
Seever Farm           2
Barrocas Farm         2
Carpentieri Farm      1
Crawford Farm         1
Batte Farm            1
Stinson Farm          1
Name: grower, dtype: int64

In [164]:
mergedDf.loc[mergedDf['grower']=='Austin Farm']

,crop_type,year,harvest_product,total_harvested,harvested_unit,field_id,org_id,cleaned_planted_area,cleaned_harvest_area,planted_area,harvested_area,grower,organization_id
269,CROP.SOYBEAN.COMMERCIAL,2015,grain,403.185902,bu,0c090e104b85e5d305640f9d43f32d0a8c5faeb6,0966e34316e39217e4f26fbe20fc14e24c26567b,57.82,57.82,57.82,57.82,Austin Farm,c9a331d4-68fb-4946-9e4e-fdca1eb268b3
270,CROP.SOYBEAN.COMMERCIAL,2015,grain,27620.687600,bu,cb1e3bd821dd673a0c378b16ba97075919aa993a,0966e34316e39217e4f26fbe20fc14e24c26567b,NaN,NaN,40.00,40.38,Austin Farm,c9a331d4-68fb-4946-9e4e-fdca1eb268b3
271,CROP.CORN.COMMERCIAL,2015,grain,41322.024805,bu,82de9e36652993a37ce5fbc55fa9033f425fb120,0966e34316e39217e4f26fbe20fc14e24c26567b,62.31,62.11,62.31,62.11,Austin Farm,c9a331d4-68fb-4946-9e4e-fdca1eb268b3
272,CROP.SOYBEAN.COMMERCIAL,2015,grain,13638.816380,bu,3043cfa778d98f7970918f0f2958c4b6e9701c7d,0966e34316e39217e4f26fbe20fc14e24c26567b,100.00,99.00,100.00,99.00,Austin Farm,c9a331d4-68fb-4946-9e4e-fdca1eb268b3
273,CROP.CORN.COMMERCIAL,2015,grain,48335.168466,bu,5ffe5f7993c967a9ec145605bb6bd6597f03b5e2,0966e34316e39217e4f26fbe20fc14e24c26567b,121.10,121.00,121.10,121.00,Austin Farm,c9a331d4-68fb-4946-9e4e-fdca1eb268b3
274,CROP.CORN.COMMERCIAL,2015,grain,102959.162415,bu,bed20ae41d9cc29734331ee52d0d94c884dc0caf,0966e34316e39217e4f26fbe20fc14e24c26567b,159.98,159.77,159.98,159.77,Austin Farm,c9a331d4-68fb-4946-9e4e-fdca1eb268b3
275,CROP.CORN.COMMERCIAL,2016,grain,127701.071130,bu,66dec081f750e2d33394a88a4436242bfbdcf6da,0966e34316e39217e4f26fbe20fc14e24c26567b,NaN,NaN,78.12,245.00,Austin Farm,c9a331d4-68fb-4946-9e4e-fdca1eb268b3
276,CROP.CORN.COMMERCIAL,2015,grain,39666.216624,bu,6444e5cd8df1b4a6d3eeb968a30822e3e486bfaa,0966e34316e39217e4f26fbe20fc14e24c26567b,NaN,NaN,42.21,42.25,Austin Farm,c9a331d4-68fb-4946-9e4e-fdca1eb268b3
277,CROP.SOYBEAN.COMMERCIAL,2015,grain,27107.456680,bu,5f826cc116841bfdbca273cebb21f470e602c75f,0966e34316e39217e4f26fbe20fc14e24c26567b,NaN,NaN,68.00,68.34,Austin Farm,c9a331d4-68fb-4946-9e4e-fdca1eb268b3
278,CROP.CORN.COMMERCIAL,2015,grain,23894.104852,bu,0dd926eb22935abce9d072635fa3301d11ca51ad,0966e34316e39217e4f26fbe20fc14e24c26567b,139.75,139.44,139.75,139.44,Austin Farm,c9a331d4-68fb-4946-9e4e-fdca1eb268b3


In [166]:
mergedDf.loc[mergedDf['grower']=='Dudley Farm'].year.value_counts()

2015    33
2014    17
Name: year, dtype: int64

#### With this I can tell that several entries are created for the same year, that could have otherwise been compacted. this can most likely be compacted using the groupby function and specificying which featured we want to see the compacted values for. 

In [168]:
mergedDf.groupby(['grower','year'])['total_harvested','cleaned_planted_area','cleaned_harvest_area'].sum()

total_harvested  cleaned_planted_area  \
grower           year                                          
Austin Farm      2015     2.038052e+06           1933.020000   
                 2016     1.614752e+05              0.000000   
Barrocas Farm    2017     1.174206e+05              0.000000   
Batte Farm       2017     7.047084e+04              0.000000   
Carpentieri Farm 2016     7.119898e+04              0.000000   
Cavanaugh Farm   2014     3.929600e+05             39.808568   
                 2015     1.589323e+06            380.285906   
                 2016     1.144137e+06            730.520000   
                 2017     1.266030e+06            519.270000   
Crawford Farm    2017     9.469835e+04              0.000000   
Depner Farm      2014     5.356606e+05           1035.128219   
                 2015     6.691972e+05            577.779999   
                 2016     2.044040e+05              0.000000   
                 2017     3.375841e+05            105.532126   
Dudley Farm      2014     8.032012e+05              0.000000   
                 2015     1.218157e+06           3452.780000   
Flanagan Farm    2015     4.109234e+04            100.630000   
                 2016     3.839162e+04             77.320000   
Gregory Farm     2015     1.991505e+05              0.000000   
                 2017     2.069695e+04              0.000000   
Hawker Farm      2015     7.485965e+04            114.750000   
                 2016     3.674980e+04            249.850000   
                 2017     6.697064e+04              0.000000   
Jenkins Farm     2016     2.321079e+05            269.497784   
Konecny Farm     2017     3.269919e+05              0.000000   
Mull Farm        2017     4.219978e+04              0.000000   
Odell Farm       2014     1.784253e+05             12.000000   
                 2016     9.191359e+04              0.000000   
                 2017     1.046427e+05             10.000000   
Overholt Farm    2017     2.207304e+05             60.000000   
Ross Farm        2017     1.619524e+05             61.720000   
Seever Farm      2017     2.226073e+05             56.000000   
Sifuentes Farm   2017     4.454520e+04              0.470000   
Simmons Farm     2015     2.582268e+06           5473.190000   
                 2016     2.221029e+06           2829.544397   
                 2017     1.738078e+06           3909.710227   
Stinson Farm     2017     8.186285e+04             10.490000   

                       cleaned_harvest_area  
grower           year                        
Austin Farm      2015           1924.080000  
                 2016             25.000000  
Barrocas Farm    2017            147.000000  
Batte Farm       2017             13.000000  
Carpentieri Farm 2016              0.000000  
Cavanaugh Farm   2014              0.000000  
                 2015           1544.010000  
                 2016            598.662117  
                 2017            100.000000  
Crawford Farm    2017            150.000000  
Depner Farm      2014            199.770000  
                 2015             12.300000  
                 2016            387.800000  
                 2017            200.000000  
Dudley Farm      2014            378.000000  
                 2015            436.700000  
Flanagan Farm    2015              0.000000  
                 2016              0.000000  
Gregory Farm     2015            309.500000  
                 2017              0.000000  
Hawker Farm      2015            114.390000  
                 2016            144.000000  
                 2017              0.000000  
Jenkins Farm     2016             93.000000  
Konecny Farm     2017            184.000000  
Mull Farm        2017            296.000000  
Odell Farm       2014            119.000000  
                 2016              0.000000  
                 2017              0.000000  
Overholt Farm    2017              0.000000  
Ross Farm        2017             58.890000  
Seever F

## It is hard to say whether the data quality is necessarily good or bad. The entries could be more secure to make the dataset more pure and so that it can be trusted with full confidence. That said, it is still possible to be able to compare valid yield data between the farms and see which ones perform the best. 